# Imports

In [18]:
import json
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

from torchinfo import summary

In [ ]:
path = "./"
root = "../"

SEED = 23

LR = 1e-3
BATCH_SIZE = 16
PRINT_ITERS = 5

In [ ]:
drive = None
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

root = root if drive is None else "/content/drive/MyDrive/crowd-counting"
path = path #######

# Data and model exploration for now

In [6]:
model = models.resnet18(weights='ResNet18_Weights.DEFAULT')

In [15]:
summary(model, (8, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [8, 1000]                 --
├─Conv2d: 1-1                            [8, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [8, 64, 112, 112]         128
├─ReLU: 1-3                              [8, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [8, 64, 56, 56]           --
├─Sequential: 1-5                        [8, 64, 56, 56]           --
│    └─BasicBlock: 2-1                   [8, 64, 56, 56]           --
│    │    └─Conv2d: 3-1                  [8, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-2             [8, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [8, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [8, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [8, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [8, 64, 56, 56]           --
│

In [11]:
for name, layer in model.named_parameters():
    print(name)

conv1.weight
bn1.weight
bn1.bias
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer2.0.conv1.weight
layer2.0.bn1.weight
layer2.0.bn1.bias
layer2.0.conv2.weight
layer2.0.bn2.weight
layer2.0.bn2.bias
layer2.0.downsample.0.weight
layer2.0.downsample.1.weight
layer2.0.downsample.1.bias
layer2.1.conv1.weight
layer2.1.bn1.weight
layer2.1.bn1.bias
layer2.1.conv2.weight
layer2.1.bn2.weight
layer2.1.bn2.bias
layer3.0.conv1.weight
layer3.0.bn1.weight
layer3.0.bn1.bias
layer3.0.conv2.weight
layer3.0.bn2.weight
layer3.0.bn2.bias
layer3.0.downsample.0.weight
layer3.0.downsample.1.weight
layer3.0.downsample.1.bias
layer3.1.conv1.weight
layer3.1.bn1.weight
layer3.1.bn1.bias
layer3.1.conv2.weight
layer3.1.bn2.weight
layer3.1.bn2.bias
layer4.0.conv1.weight
layer4.0.bn1.weight
layer4.0.bn1.bias
layer4.0.conv2.we

# Data——NWPU-Crowd (to-download)

## 5,109 images, 2,133,375 annotated instances with point and box labels
### [1-3109] train, [3110-3609] val, [3610-5109] test

### .txt files include the [index, luminance label, scene level]
#### luminance = 3 classes in YUV color space: [0, 0.25], (0.25, 0.5],and (0.5, 0.75]
#### scene level = 5 classes based on num_people: 0, (0, 100], (100, 500], (500, 5000]

## i) Labels

In [57]:
train_labels = pd.read_csv('data/train.txt', sep=" ", header=None)
val_labels = pd.read_csv('data/val.txt', sep=" ", header=None)
test_labels = pd.read_csv('data/test.txt', sep=" ", header=None)

In [58]:
def prep_labels(df):
    df = df.drop(columns=0)
    df = df.rename(columns={1: 'luminance', 2: 'scene_level'})
    return df

In [59]:
train_labels = prep_labels(train_labels)
val_labels = prep_labels(val_labels)
test_labels = prep_labels(test_labels)

# not done. read json files to merge just "img_id" w/ df index and take "human_num"